In [ ]:
# !conda update conda -y
# !conda update conda -y
# !conda install psycopg2 -y

In [1]:
import psycopg2
from psycopg2 import errorcodes, errors
import os, sys, getpass

In [2]:
getpass.getuser()

'seanreed1'

In [3]:
conn = psycopg2.connect(database="jupyterdb", user=getpass.getuser())


In [4]:
cur = conn.cursor()
cur.execute("SELECT 1;")
conn.commit()
cur.fetchall()

[(1,)]

Note: If you don't want to have **conn.commit()** everywhere in your code, set

conn.autocommit = True

In [5]:
# let's cleanup
cur.close()
conn.close()

In [6]:
# what if we get the database name wrong?
conn = psycopg2.connect(database="doesnotexist", user=getpass.getuser())

OperationalError: FATAL:  database "doesnotexist" does not exist


In [ ]:
## let's catch the Operational Error for db does not exist now

https://www-freecodecamp-org.cdn.ampproject.org/c/s/www.freecodecamp.org/news/connect-python-with-sql/amp/

https://kb.objectrocket.com/postgresql/python-error-handling-with-the-psycopg2-postgresql-adapter-645

In [ ]:
# # define a function that handles and parses psycopg2 exceptions
# MIGHT NOT NEED THIS ANYMORE
# def print_psycopg2_exception(err):
#     # get details about the exception
#     err_type, err_obj, traceback = sys.exc_info()

#     # get the line number when exception occured
#     line_num = traceback.tb_lineno
#     # print the connect() error
#     print (f"\npsycopg2 ERROR: {err}, on line number:{line_num}\n")
#     print (f"psycopg2 traceback: {traceback}\ntype: {err_type}\n")

#     # psycopg2 extensions.Diagnostics object attribute
#     print (f"\nextensions.Diagnostics: {err.diag}")

#     # print the pgcode and pgerror exceptions
#     print (f"pgerror: {err.pgerror}\n")
#     print (f"pgcode: {err.pgcode} \n")

In [11]:
def create_server_connection(database, user):
    conn = None
    try:
        conn = psycopg2.connect(
            database=database,
            user=user,
        )
        print(f"Connected to Database {database} successfully")
        
    except Exception as e:
        print(e)

    return conn

In [12]:
conn = create_server_connection(database="doesnotexist", user=getpass.getuser())

FATAL:  database "doesnotexist" does not exist



In [ ]:
#let's make sure it can still connect normally

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = conn.cursor()
cur.execute("SELECT 1;")
conn.commit()
cur.fetchall()

In [ ]:
# let's cleanup

cur.close()
conn.close()

The function *print_psycopg2_exception* also using the psycopg.errors module to catch postgresql errors.

In [ ]:
dir(errors) #psycopg2 errors

In [ ]:
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())


In [ ]:
# Let's check error handling on SQL statements WITHOUT OUR FUNCTION ABOVE
# Spoiler: we only get a syntax error

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())

with conn.cursor() as cursor:
    try:
        cursor.execute("SELECT * FROM NOTABLE;")
        conn.commit()
    except Exception as e:
        print(e)
    finally:
        conn.close()


In [ ]:
# Let's check error handling on SQL statements using print_psycopg2_exception()
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
with conn.cursor() as cursor:
    try:
        cursor.execute("SELECT * FROM NOTABLE;")
        conn.commit()
    except Exception as e:
        # pass exception to function
        print_psycopg2_exception(e)
    finally:
        conn.close()

In [ ]:
conn.close()

In [ ]:
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())

In [ ]:
def create_database(conn, query):
    if conn is not None:
        cur = conn.cursor()
        try:
            cur.execute(query)
            print("Database created successfully")
        except Exception as e:
            print(f"Error: '{e}'")

In [ ]:
mysql
# def create_database(connection, query):
#     cursor = connection.cursor()
#     try:
#         cursor.execute(query)
#         print("Database created successfully")
#     except Error as err:
#         print(f"Error: '{err}'")